# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is **to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund**.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

NumPy is faster than nrmal because is designed in C++ and in finance has more speed, so is naturally faster.
Pandas is the abbreviation of "panel data", we already know it.



In [15]:
import numpy as np
import pandas as pd
import requests #The gold-standard for http-request, this is requests to the internet to get back some data.
import xlsxwriter   #To write Excel document from Python Scritps.
import math

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

---

In the real world you will connect to an API or an Index Provider so that if a stock is added or removed from the SP500 my equal weight reflect that. All the tools that provide that are paid so we will not be using it.

In [16]:
stocks = pd.read_csv("sp_500_stocks.csv")
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


As there are more than 500 some companies are duplicated.

## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

---
We will be doing it in **sandbox mode** that is like a play mode, it returns like the real API, financial data but do not return it in real time. Instead, it returns randomized financial data.

We use it to see if our function works properly.

---

**API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository**. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [17]:
from api_secrets import IEX_CLOUD_API_TOKEN
IEX_CLOUD_API_TOKEN

'sk_716f447c7a36417a96ee0a514341b523'

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock

---

We need:

- A base **URL** that appears in the documentation of the API that we are working. We can find it in our case writting "iex cloud api" in the browser. To locate it, we need to find a block that has written "API Reference". In our case, we are using the Sandbox mode so we cannot use that one.

- We will do a *"Ctrl + F"* and look for *sandbox*, and where is written "The base URL is", you copy it and paste below:
- Then, we need to add the "Quote" to figure out the endpoint that provides market cap and price of each stock.
- We use the token to identify to the API.
- We store the data in a variable with the library "request".

In [18]:
# First, we do it with one stock.
symbol = "AAPL" #Apple
api_url = f'https://api.iex.cloud/v1/data/core/quote/{symbol}?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()[0]
print(data)

{'avgTotalVolume': 65017399, 'calculationPrice': 'tops', 'change': 0.47, 'changePercent': 0.00252, 'close': None, 'closeSource': 'official', 'closeTime': None, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': None, 'delayedPriceTime': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPrice': None, 'extendedPriceTime': None, 'high': None, 'highSource': None, 'highTime': None, 'iexAskPrice': 189, 'iexAskSize': 285, 'iexBidPrice': 186.74, 'iexBidSize': 100, 'iexClose': 186.75, 'iexCloseTime': 1715695872053, 'iexLastUpdated': 1715695872053, 'iexMarketPercent': 0.011521068710016449, 'iexOpen': 187.64, 'iexOpenTime': 1715693400488, 'iexRealtimePrice': 186.75, 'iexRealtimeSize': 100, 'iexVolume': 146837, 'lastTradeTime': 1715695872053, 'latestPrice': 186.75, 'latestSource': 'IEX real time price', 'latestTime': '10:11:12 AM', 'latestUpdate': 1715695872053, 'latestVolume': None, 'low': None, 'lowSource': 'IEX real time price', 'lowTime': 1715695849997, 'marketC

We see that it returns an error.

If we look into the Error Codes, we see this:
Error Codes

IEX Cloud uses HTTP response codes to indicate the success or failure of an API request.
General HTML status codes

- 2xx Success.
- 4xx Errors based on information provided in the request
- 5xx Errors on IEX Cloud servers

So something is wrong in our request. We find that 404 means or Not Found or Unknown Symbol, in our case is the second one. 

If we go in the documentation with a *Ctrl+F* we will find a category called **Name Convention** and there, we find that we need to specify that we want **stable**.

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [19]:
price = data['latestPrice']
market_cap = data['marketCap']

## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

---

Before we start adding values to the DataFrame, we need to have crystal clear how do we want the shape of our DataFrame. It is going to have:

- The ticker for each stock.
- The market capitalization for each stock.
- The number of shares to buy from each stock.


In [20]:
my_columns = ["Ticker", "Stock Price", "Market Capitalization", "Number of Shares to Buy"]
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe


,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy


In [21]:
final_dataframe.loc[len(final_dataframe.index)] = [symbol,price,market_cap,'N/A'] 

In [22]:
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,AAPL,186.75,2863639813500,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [23]:
final_dataframe = pd.DataFrame(columns=my_columns)
for stock in stocks["Ticker"][:5]:
    api_url = f'https://api.iex.cloud/v1/data/core/quote/{stock}?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()[0]
    final_dataframe.loc[len(final_dataframe.index)] = [stock,data['latestPrice'],data["marketCap"],'N/A'] 


In [24]:
# Prueba
"""
final_dataframe = pd.DataFrame(columns=my_columns)
for stock in stocks["Ticker"][:5]:
    api_url = f'https://api.iex.cloud/v1/data/core/quote/{stock}?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    #final_dataframe.loc[len(final_dataframe.index)] = [stock,data['latestPrice'],data["marketCap"],'N/A'] 

data
"""

'\nfinal_dataframe = pd.DataFrame(columns=my_columns)\nfor stock in stocks["Ticker"][:5]:\n    api_url = f\'https://api.iex.cloud/v1/data/core/quote/{stock}?token={IEX_CLOUD_API_TOKEN}\'\n    data = requests.get(api_url).json()\n    #final_dataframe.loc[len(final_dataframe.index)] = [stock,data[\'latestPrice\'],data["marketCap"],\'N/A\'] \n\ndata\n'

## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [25]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [26]:
"""
symbol_groups = list(chunks(stocks["Ticker"], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
final_dataframe = pd.DataFrame(columns=my_columns)
"""

#symbol_groups[0]   #Listas separadas en grupos
#symbol_strings[0]   #Los lee letra por letra

# Leer las posiciones del array
"""
for j in range(0,len(symbol_groups)):
    for k in range(0,len(symbol_groups[j])):
        print(k)
"""
# Leer los Tickers de las compañías del índice
"""
for j in range(0,len(symbol_groups)):
    for k in symbol_groups[j]:
        print(k)
"""

#Lista donde se almacenan los datos en 6 casillas con 100 elementos, excepto la última con 5.
"""
lista_datos=[]
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://api.iex.cloud/v1/data/core/QUOTE/{symbol_string}?last=2&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    lista_datos.append(data)
"""
"""
print(len(lista_datos))     
print(len(lista_datos[0]))

lista_datos[0][0]["latestPrice"]
"""

# Añadir lastestPrice y marketCap
"""
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://api.iex.cloud/v1/data/core/QUOTE/{symbol_string}?last=2&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for j in range(0,len(symbol_groups)):
        for k in range(0,len(symbol_groups[j])):
            final_dataframe.loc[len(final_dataframe.index)] = ["Nombre", data[j][k]['latestPrice'],data[j][k]['marketCap'],'N/A']
"""
            

# No avanza el sybol string a la vez que j, que es lo que necesitamos.
"""
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://api.iex.cloud/v1/data/core/QUOTE/{symbol_string}?last=2&token={IEX_CLOUD_API_TOKEN}'
    for j in range(0,len(symbol_groups)):
        data = requests.get(batch_api_call_url).json()
        for k in range(0,len(symbol_groups[0])):
            #print(data[k]["symbol"], data[k]['latestPrice'],data[k]['marketCap'])
            final_dataframe.loc[len(final_dataframe.index)] = [data[k]["symbol"], data[k]['latestPrice'],data[k]['marketCap'],'N/A']

final_dataframe    
"""
"""
    for j in range(0,1):
        for k in range(0,len(symbol_groups[0])):
            final_dataframe.loc[len(final_dataframe.index)] = ["Nombre", data[j][k]['latestPrice'],data[j][k]['marketCap'],'N/A']
"""

'\n    for j in range(0,1):\n        for k in range(0,len(symbol_groups[0])):\n            final_dataframe.loc[len(final_dataframe.index)] = ["Nombre", data[j][k][\'latestPrice\'],data[j][k][\'marketCap\'],\'N/A\']\n'

In [27]:
symbol_groups = list(chunks(stocks["Ticker"], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
final_dataframe = pd.DataFrame(columns=my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://api.iex.cloud/v1/data/core/QUOTE/{symbol_string}?last=2&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for position in range(0,len(symbol_string.split(','))):
        final_dataframe.loc[len(final_dataframe.index)] = [data[position]["symbol"], data[position]['latestPrice'],data[position]['marketCap'],'N/A']

final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,150.865,44211785421,N/A
1,AAL,15.335,10061145425,N/A
2,AAP,76.670,4570258142,N/A
3,AAPL,186.740,2863486472680,N/A
4,ABBV,161.120,284516616875,N/A
...,...,...,...,...
500,YUM,137.770,38800469847,N/A
501,ZBH,120.110,24710037884,N/A
502,ZBRA,321.860,16549849050,N/A
503,ZION,44.500,6570635574,N/A


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

---

When we enter a value we are looking for it to be only a numeric value. So, what we are going to do is to make it distinguish between ValueErrors.

In [28]:
portfolio_size = input('Enter the value of your portfolio: ')

try:
    val = float(portfolio_size)
except ValueError:
    print('That´s not a number! Please try again.')
    portfolio_size = input('Enter the value of your portfolio: ')
    val = float(portfolio_size)

In [29]:
position_size = val/len(final_dataframe.index)   #Value divided for each share.

for i in range(0,len(final_dataframe)):
    final_dataframe["Number of Shares to Buy"][i] = math.floor(position_size/ final_dataframe["Stock Price"][i])
final_dataframe

C:\Users\Manu\AppData\Local\Temp\ipykernel_15268\3896938942.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_dataframe["Number of Shares to Buy"][i] = math.floor(position_size/ final_dataframe["Stock Price"][i])
C:\Users\Manu\AppData\Local\Temp\ipykernel_15268\3896938942.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_dataframe["Number of Shares to Buy"][i] = math.floor(position_size/ final_dataframe["Stock Price"][i])
C:\Users\Manu\AppData\Local\Temp\ipykernel_15268\3896938942.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the doc

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,150.865,44211785421,131
1,AAL,15.335,10061145425,1291
2,AAP,76.670,4570258142,258
3,AAPL,186.740,2863486472680,106
4,ABBV,161.120,284516616875,122
...,...,...,...,...
500,YUM,137.770,38800469847,143
501,ZBH,120.110,24710037884,164
502,ZBRA,321.860,16549849050,61
503,ZION,44.500,6570635574,444


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [34]:
writer = pd.ExcelWriter('recommended trades.xlsx', engine = 'xlsxwriter') #We need to specify because it can save in XML.
final_dataframe.to_excel(writer, 'Recommended Trades', index = False)
#We use pandas due to the capacity to manipulate tabular data

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [35]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

dollar_format = writer.book.add_format(
    {
        'num_format':'$0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

integer_format = writer.book.add_format(
    {
        'num_format':'0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)


### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```
---

This is for applyting the formats that we have created above. If it returns 0 means that it is with good syntax. Too see it in action you need to save your writer object.

In [36]:
"""
writer.sheets['Recommended Trades'].set_column('A:A',18,string_format)
writer.sheets['Recommended Trades'].set_column('B:B',18,string_format)
writer.sheets['Recommended Trades'].set_column('C:C',18,string_format)
writer.sheets['Recommended Trades'].set_column('D:D',18,string_format)

writer.close()
"""

writer.sheets["Recommended Trades"].write("A1","Ticker",string_format)
writer.sheets["Recommended Trades"].write("B1","Stock Price",dollar_format)
writer.sheets["Recommended Trades"].write("C1","Market Capitalization",dollar_format)
writer.sheets["Recommended Trades"].write("D1","Number of Shares to Buy",integer_format)

0

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

In [38]:
column_formats = {
    'A':["Ticker",string_format],
    'B':["Stock Price",dollar_format],
    'C':["Market Capitalization",dollar_format],
    'D':["Number of Shares to Buy",integer_format]
}

for column in column_formats.keys():
    writer.sheets["Recommended Trades"].set_column(f'{column}:{column}',18, column_formats[column][1])
    writer.sheets["Recommended Trades"].write(f'{column}1',column_formats[column][0], column_formats[column][1])


## Saving Our Excel Output

Saving our Excel file is very easy:

In [39]:
writer.close()

c:\Users\Manu\AppData\Local\Programs\Python\Python312\Lib\site-packages\xlsxwriter\workbook.py:369: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")
